In [13]:
import os
from imutils import paths
from xml.etree import ElementTree as ET
import shutil

if not os.path.isdir('cleaned'):
                os.mkdir('cleaned')

xmlPaths = sorted(list(paths.list_files('train')))
testPaths = sorted(list(paths.list_files('test')))

for xmlpath in xmlPaths:
    file_name, file_extension = os.path.splitext(xmlpath)
    if(file_extension == '.xml'):
        with open(xmlpath, 'r') as f:
            data = f.read()
            value = ET.fromstring(data).find('object/name').text
            
            if not os.path.isdir('cleaned/' + value):
                os.mkdir('cleaned/' + value)
            shutil.copy(file_name + '.jpg','cleaned/' + value)

for xmlpath in testPaths:
    file_name, file_extension = os.path.splitext(xmlpath)
    if(file_extension == '.xml'):
        with open(xmlpath, 'r') as f:
            data = f.read()
            value = ET.fromstring(data).find('object/name').text
            shutil.copy(file_name + '.jpg','cleaned/' + value)


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers.core import Dense
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers.core import activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras import backend as K
from keras.layers import Softmax
import scipy.io
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.utils import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from imutils import paths
import numpy as np
import random
import cv2
import os
import pickle
import math

class VGG16:

    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        model.add(Conv2D(input_shape=inputShape, filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(GlobalAveragePooling2D())
        model.add(Flatten())
        model.add(Dense(units=2048,activation="relu"))
        model.add(Dropout(0.2))
        model.add(Dense(units=256,activation="relu"))
        model.add(Dense(units=classes, activation="softmax"))

        return model

    

data = []
labels = []
imagePaths = sorted(list(paths.list_images('cleaned')))
random.seed(42)
random.shuffle(imagePaths)

EPOCHS = 5
INIT_LR = 1e-4
BS = 32
IMAGE_DIMS = (150, 150, 3)

for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)

	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
	
data = np.array(data, dtype = "float32")/255.0
labels = np.array(labels)
print("[INFO] datamatrix: {:.2f}MB".format(data.nbytes / (1024 * 1000)))
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

print("[INFO] serializing label binarizer...")
f = open('lb', "wb")
f.write(pickle.dumps(lb))
f.close()    

print("[INFO] compiling model...")
model = VGG16.build(width = IMAGE_DIMS[1], height = IMAGE_DIMS[0], depth = IMAGE_DIMS[2], classes =  len(lb.classes_))
opt = Adam(learning_rate = INIT_LR,  decay = INIT_LR/80)
model.compile(loss = "categorical_crossentropy",optimizer = opt, metrics = ["accuracy"])

mcp_save = ModelCheckpoint('mmmm.h5', save_best_only=True)

print("[INFO] training network...")
H = model.fit(data, labels,epochs = EPOCHS, batch_size= BS, validation_split=0.3,  callbacks=[mcp_save])

[INFO] datamatrix: 338.82MB
[INFO] serializing label binarizer...
[INFO] compiling model...
[INFO] training network...
Epoch 1/5
29/29 [==============================] - 353s 12s/step - loss: 1.5853 - accuracy: 0.2547 - val_loss: 1.5737 - val_accuracy: 0.2435
Epoch 2/5
29/29 [==============================] - 355s 12s/step - loss: 1.5598 - accuracy: 0.2870 - val_loss: 1.5731 - val_accuracy: 0.2435
Epoch 3/5
29/29 [==============================] - 357s 12s/step - loss: 1.5237 - accuracy: 0.2747 - val_loss: 1.4325 - val_accuracy: 0.3808
Epoch 4/5
29/29 [==============================] - 354s 12s/step - loss: 1.3019 - accuracy: 0.4227 - val_loss: 1.1047 - val_accuracy: 0.5078
Epoch 5/5
29/29 [==============================] - 358s 12s/step - loss: 1.1272 - accuracy: 0.5139 - val_loss: 1.0292 - val_accuracy: 0.5725


In [4]:
model.save("m111.h5")

In [5]:
import tensorflow as tf


#mod = tf.keras.models.load_model(model)

mod =tf.keras.models.load_model("mmmm.h5")

converter=tf.lite.TFLiteConverter.from_keras_model(mod)

tflite_model=converter.convert()

with open('FYP-M1.tflite', 'wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\shahe\AppData\Local\Temp\tmpm1smxv9q\assets


INFO:tensorflow:Assets written to: C:\Users\shahe\AppData\Local\Temp\tmpm1smxv9q\assets


In [6]:
import tensorflow as tf


#mod = tf.keras.models.load_model(model)

mod =tf.keras.models.load_model("m111.h5")

converter=tf.lite.TFLiteConverter.from_keras_model(mod)

tflite_model=converter.convert()

with open('FYP-M2.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\shahe\AppData\Local\Temp\tmpffjebtal\assets


INFO:tensorflow:Assets written to: C:\Users\shahe\AppData\Local\Temp\tmpffjebtal\assets


In [8]:
from keras.models import load_model

IMAGE_DIMS = (112, 112, 3)

print("[INFO] loading images...")
testpaths = sorted(list(paths.list_images('test')))
model = load_model('model.h5')
lb = pickle.loads(open('lb', "rb").read())


for imagePath in testpaths:

    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = image.astype("float") / 255.0
    img_to_array(image)
    image = np.expand_dims(image, axis = 0)
    predict = model.predict(image)
    print(predict)
    proba = np.array(predict[0])
    print(proba)
    idx = np.argmax(proba)
    #print(idx)
    label = lb.classes_[idx]
    file_name = os.path.basename(imagePath)
    print(file_name)
    print("[INFO] {}".format(label))


[INFO] loading images...
1/1 [==============================] - 0s 116ms/step
[[8.8876128e-05 9.9954814e-01 3.6179487e-04 2.9339706e-08 1.1491185e-06]]
[8.8876128e-05 9.9954814e-01 3.6179487e-04 2.9339706e-08 1.1491185e-06]
ezgif-frame-003_jpg.rf.6dbec6452f172d8dda92c2b0fa4895d5.jpg
[INFO] Azadiractha Indica
1/1 [==============================] - 0s 73ms/step
[[7.6646349e-05 9.9954635e-01 3.7590505e-04 2.1698668e-08 1.1095360e-06]]
[7.6646349e-05 9.9954635e-01 3.7590505e-04 2.1698668e-08 1.1095360e-06]
ezgif-frame-004_jpg.rf.87884e4a07e9b726cf994e5caeece92a.jpg
[INFO] Azadiractha Indica
1/1 [==============================] - 0s 72ms/step
[[5.4214173e-04 9.9659663e-01 2.8537959e-03 4.4527724e-07 7.0546012e-06]]
[5.4214173e-04 9.9659663e-01 2.8537959e-03 4.4527724e-07 7.0546012e-06]
ezgif-frame-016_jpg.rf.aa67d5906eebbcd177293416a4139c26.jpg
[INFO] Azadiractha Indica
1/1 [==============================] - 0s 72ms/step
[[1.2443905e-01 4.3541920e-07 6.5692037e-04 8.7332320e-01 1.5804371e-0

1/1 [==============================] - 0s 75ms/step
[[8.1040221e-01 9.0226822e-06 3.3360984e-02 1.4526701e-01 1.0960820e-02]]
[8.1040221e-01 9.0226822e-06 3.3360984e-02 1.4526701e-01 1.0960820e-02]
ezgif-frame-055_jpg.rf.455f33b03708e81b1915f9f19e5f3d2e.jpg
[INFO] Azadirachta Indica-Neem-
1/1 [==============================] - 0s 73ms/step
[[8.1544822e-05 9.9972671e-01 1.9149507e-04 1.9948599e-08 2.7919407e-07]]
[8.1544822e-05 9.9972671e-01 1.9149507e-04 1.9948599e-08 2.7919407e-07]
ezgif-frame-055_jpg.rf.aa4262943f92edff92877d39866e70a2.jpg
[INFO] Azadiractha Indica
1/1 [==============================] - 0s 78ms/step
[[1.5846052e-03 1.0711893e-02 9.8726034e-01 7.9544607e-06 4.3523544e-04]]
[1.5846052e-03 1.0711893e-02 9.8726034e-01 7.9544607e-06 4.3523544e-04]
ezgif-frame-056_jpg.rf.5c4a89ec20921f07a381a841d3784382.jpg
[INFO] Calotropis
1/1 [==============================] - 0s 76ms/step
[[9.7924203e-04 4.6445816e-09 1.5542757e-04 4.3159729e-05 9.9882215e-01]]
[9.7924203e-04 4.6445816

1/1 [==============================] - 0s 73ms/step
[[1.2873282e-01 2.4187524e-05 3.1931061e-02 5.0256822e-02 7.8905517e-01]]
[1.2873282e-01 2.4187524e-05 3.1931061e-02 5.0256822e-02 7.8905517e-01]
ezgif-frame-107_jpg.rf.3ae95784fb16946618bb232e33b4c0b0.jpg
[INFO] Oleander
1/1 [==============================] - 0s 73ms/step
[[3.6551240e-01 1.2244768e-06 1.5243644e-02 9.5642999e-02 5.2359968e-01]]
[3.6551240e-01 1.2244768e-06 1.5243644e-02 9.5642999e-02 5.2359968e-01]
ezgif-frame-114_jpg.rf.169958f64e051622d7a8e9974b21f3d6.jpg
[INFO] Oleander
1/1 [==============================] - 0s 71ms/step
[[1.4767566e-01 2.9262091e-04 8.0649316e-01 1.5118444e-02 3.0420164e-02]]
[1.4767566e-01 2.9262091e-04 8.0649316e-01 1.5118444e-02 3.0420164e-02]
ezgif-frame-116_jpg.rf.61e1b06583789e336a220edd913677c6.jpg
[INFO] Calotropis
1/1 [==============================] - 0s 77ms/step
[[8.2732117e-01 1.1292904e-07 2.4382186e-03 1.7022753e-01 1.3034535e-05]]
[8.2732117e-01 1.1292904e-07 2.4382186e-03 1.70227

In [3]:
import tensorflow as tf


#mod = tf.keras.models.load_model(model)

mod =tf.keras.models.load_model("model.h5")

converter=tf.lite.TFLiteConverter.from_keras_model(mod)

tflite_model=converter.convert()

with open('M2.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\shahe\AppData\Local\Temp\tmp0fi3mw9e\assets


INFO:tensorflow:Assets written to: C:\Users\shahe\AppData\Local\Temp\tmp0fi3mw9e\assets


In [37]:
import tensorflow as tf

from tensorflow.keras.utils import load_img, img_to_array 
from time import time 
from matplotlib import pyplot as plt 
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers.core import Dense
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers.core import activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras import backend as K
from keras.layers import Softmax
import scipy.io
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.utils import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from imutils import paths
import numpy as np
import random
import cv2
import os
import pickle
import math


#IMAGE_DIMS = (150, 150, 3)

imagePath="ezgif-frame-079_jpg.rf.3de8c2b316fdd78f9507885e65bb5b6e.jpg"
image = cv2.imread(imagePath)
image = cv2.resize(image, (112, 112))
#image = image.astype("float32") / 255.0
image = img_to_array(image)
image=np.float32(image)
image = np.expand_dims(image, axis = 0)

#image = normalize(image, axis=1)


tflite_model_path = "model.tflite"

interpreter = tf.lite.Interpreter(model_path=tflite_model_path) 
interpreter.allocate_tensors()

input_details = interpreter.get_input_details() 
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

input_type = input_details[0]['dtype']


print(input_type)

print(image.shape)

print(input_shape)
print(output_shape)


input_data = image
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data_tflite = interpreter.get_tensor(output_details[0]['index'])
print(output_data_tflite)


#mod =tf.keras.models.load_model("model.h5")
#predict = mod.predict(image)
#print(predict)



#[[6.5081363e-04 1.1774694e-08 1.0312422e-04 3.9550650e-05 9.9920648e-01]]
#[[7.6324429e-04 1.1551837e-09 3.0232544e-05 7.9875943e-05 9.9912661e-01]] 73

#[[8.8876128e-05 9.9954814e-01 3.6179487e-04 2.9339706e-08 1.1491185e-06]] 03
#[[1.8437373e-09 1.0000000e+00 5.0801610e-08 1.1184435e-16 4.6766284e-12]]


<class 'numpy.float32'>
(1, 112, 112, 3)
[  1 112 112   3]
[1 5]
[[1.6551854e-09 1.0000000e+00 3.2838109e-08 8.7938127e-17 2.4816742e-12]]
